In [1]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN
from keras import layers

C:\Users\charlychiu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 256
LAYERS = 1

### one-hot encoding converter

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

### one-hot encoding map

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

## Data Generation

- addition data: 80000
- substraction data: 80000
- total data: 160000
- length of the sequence: 7

In [7]:
questions_with_plus = []
expected_with_plus = []
seen_with_plus = set()
print('Generating data...')
while len(questions_with_plus) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen_with_plus:
        continue
    seen_with_plus.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions_with_plus.append(query)
    expected_with_plus.append(ans)
print('Total addition questions:', len(questions_with_plus))

Generating data...
Total addition questions: 80000


### addition data sample

In [8]:
print(questions_with_plus[:5], expected_with_plus[:5])

['7+0    ', '7+4    ', '3+0    ', '965+350', '7+779  '] ['7   ', '11  ', '3   ', '1315', '786 ']


In [9]:
questions_with_minus = []
expected_with_minus = []
seen_with_minus = set()
print('Generating data...')
while len(questions_with_minus) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if a < b:
        continue
    if key in seen_with_minus:
        continue
    seen_with_minus.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions_with_minus.append(query)
    expected_with_minus.append(ans)
print('Total subtraction questions:', len(questions_with_minus))

Generating data...
Total subtraction questions: 80000


### subtraction data sample

In [10]:
print(questions_with_minus[:5], expected_with_minus[:5])

['47-5   ', '58-9   ', '815-1  ', '89-20  ', '901-843'] ['42  ', '49  ', '814 ', '69  ', '58  ']


### combine addtion & subtraction data

In [11]:
questions = questions_with_plus + questions_with_minus
expected = expected_with_plus + expected_with_minus

## Processing

### transfer data to one-hot representation

In [12]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


### Split data into training, validation, testing

In [13]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:40000]
train_y = y[:40000]
test_x = x[40000:]
test_y = y[40000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(36000, 7, 13)
(36000, 4, 13)
Validation Data:
(4000, 7, 13)
(4000, 4, 13)
Testing Data:
(120000, 7, 13)
(120000, 4, 13)


In [14]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False False  True False False False False False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [ True False False False False False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False  True False False False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False  True False False Fals

## Build Model

In [15]:
def get_model():
    model = Sequential()
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    model.add(layers.RepeatVector(DIGITS + 1))
    for _ in range(LAYERS):
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
print('Build model...')
model = get_model()
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


### Training

In [33]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 102us/step - loss: 9.6674e-04 - acc: 1.0000 - val_loss: 0.0625 - val_acc: 0.9817
Q 394-242 T 152  ☑ 152 
Q 364-42  T 322  ☑ 322 
Q 51+116  T 167  ☑ 167 
Q 555-451 T 104  ☑ 104 
Q 88+797  T 885  ☑ 885 
Q 421+490 T 911  ☑ 911 
Q 128-2   T 126  ☑ 126 
Q 413+195 T 608  ☑ 608 
Q 613-4   T 609  ☑ 609 
Q 56+298  T 354  ☑ 354 

--------------------------------------------------
Iteration 1
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 104us/step - loss: 8.5962e-04 - acc: 1.0000 - val_loss: 0.0623 - val_acc: 0.9822
Q 863-7   T 856  ☑ 856 
Q 819-71  T 748  ☑ 748 
Q 170-116 T 54   ☒ 53  
Q 889-99  T 790  ☑ 790 
Q 343-72  T 271  ☑ 271 
Q 141+16  T 157  ☑ 157 
Q 879-13  T 866  ☑ 866 
Q 916-48  T 868  ☑ 868 
Q 317+282 T 599  ☑ 599 
Q 490-356 T 134  ☑ 134 

-----------

36000/36000 [==============================] - 4s 103us/step - loss: 4.6656e-04 - acc: 1.0000 - val_loss: 0.0596 - val_acc: 0.9820
Q 852-82  T 770  ☑ 770 
Q 784+26  T 810  ☑ 810 
Q 243+91  T 334  ☑ 334 
Q 456-128 T 328  ☑ 328 
Q 859+83  T 942  ☑ 942 
Q 689+55  T 744  ☑ 744 
Q 944+631 T 1575 ☑ 1575
Q 667-28  T 639  ☑ 639 
Q 451-243 T 208  ☑ 208 
Q 469+2   T 471  ☑ 471 

--------------------------------------------------
Iteration 15
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 102us/step - loss: 4.5066e-04 - acc: 1.0000 - val_loss: 0.0602 - val_acc: 0.9825
Q 36+822  T 858  ☑ 858 
Q 318+986 T 1304 ☑ 1304
Q 323-7   T 316  ☑ 316 
Q 884+65  T 949  ☑ 949 
Q 956-635 T 321  ☑ 321 
Q 905-90  T 815  ☑ 815 
Q 710-378 T 332  ☑ 332 
Q 404-81  T 323  ☑ 323 
Q 145+956 T 1101 ☑ 1101
Q 882-9   T 873  ☑ 873 

--------------------------------------------------
Iteration 16
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/3600

36000/36000 [==============================] - 4s 105us/step - loss: 7.1848e-04 - acc: 1.0000 - val_loss: 0.0566 - val_acc: 0.9826
Q 823+12  T 835  ☑ 835 
Q 650+3   T 653  ☑ 653 
Q 288-57  T 231  ☑ 231 
Q 9+564   T 573  ☑ 573 
Q 25+75   T 100  ☒ 90  
Q 718+342 T 1060 ☑ 1060
Q 75+842  T 917  ☑ 917 
Q 909-606 T 303  ☑ 303 
Q 436-0   T 436  ☑ 436 
Q 863+33  T 896  ☑ 896 

--------------------------------------------------
Iteration 29
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 106us/step - loss: 6.7146e-04 - acc: 1.0000 - val_loss: 0.0566 - val_acc: 0.9829
Q 943+70  T 1013 ☑ 1013
Q 761-0   T 761  ☑ 761 
Q 723+3   T 726  ☑ 726 
Q 66+233  T 299  ☑ 299 
Q 997-741 T 256  ☑ 256 
Q 733-91  T 642  ☑ 642 
Q 209-25  T 184  ☑ 184 
Q 873-820 T 53   ☑ 53  
Q 71-29   T 42   ☑ 42  
Q 302-65  T 237  ☑ 237 

--------------------------------------------------
Iteration 30
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/3600

36000/36000 [==============================] - 4s 104us/step - loss: 3.9790e-04 - acc: 1.0000 - val_loss: 0.0565 - val_acc: 0.9826
Q 557-57  T 500  ☑ 500 
Q 386+63  T 449  ☑ 449 
Q 72+575  T 647  ☑ 647 
Q 85+573  T 658  ☑ 658 
Q 131+17  T 148  ☑ 148 
Q 25+54   T 79   ☒ 89  
Q 9+777   T 786  ☑ 786 
Q 637-513 T 124  ☑ 124 
Q 76-63   T 13   ☑ 13  
Q 26+173  T 199  ☒ 299 

--------------------------------------------------
Iteration 43
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 101us/step - loss: 3.8497e-04 - acc: 1.0000 - val_loss: 0.0569 - val_acc: 0.9828
Q 148+417 T 565  ☑ 565 
Q 49-2    T 47   ☑ 47  
Q 715+20  T 735  ☑ 735 
Q 650-629 T 21   ☒ 10  
Q 668+385 T 1053 ☑ 1053
Q 251+681 T 932  ☑ 932 
Q 225+307 T 532  ☒ 533 
Q 398-297 T 101  ☒ 111 
Q 572-81  T 491  ☑ 491 
Q 1+58    T 59   ☑ 59  

--------------------------------------------------
Iteration 44
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/3600

36000/36000 [==============================] - 4s 101us/step - loss: 6.2398e-04 - acc: 1.0000 - val_loss: 0.0587 - val_acc: 0.9836
Q 446+47  T 493  ☑ 493 
Q 850+813 T 1663 ☑ 1663
Q 320-25  T 295  ☑ 295 
Q 63-33   T 30   ☑ 30  
Q 81-11   T 70   ☒ 60  
Q 229-9   T 220  ☑ 220 
Q 73+830  T 903  ☑ 903 
Q 139-124 T 15   ☒ 25  
Q 494+79  T 573  ☑ 573 
Q 80+54   T 134  ☑ 134 

--------------------------------------------------
Iteration 57
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 104us/step - loss: 5.8471e-04 - acc: 1.0000 - val_loss: 0.0584 - val_acc: 0.9834
Q 374+118 T 492  ☑ 492 
Q 61+12   T 73   ☑ 73  
Q 617-394 T 223  ☑ 223 
Q 127-8   T 119  ☑ 119 
Q 372-59  T 313  ☑ 313 
Q 451-243 T 208  ☑ 208 
Q 794-743 T 51   ☑ 51  
Q 54+346  T 400  ☑ 400 
Q 31+252  T 283  ☑ 283 
Q 452-448 T 4    ☒ 5   

--------------------------------------------------
Iteration 58
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/3600

36000/36000 [==============================] - 4s 106us/step - loss: 3.4761e-04 - acc: 1.0000 - val_loss: 0.0580 - val_acc: 0.9840
Q 531-54  T 477  ☑ 477 
Q 484+42  T 526  ☑ 526 
Q 967-603 T 364  ☑ 364 
Q 86-31   T 55   ☑ 55  
Q 652-194 T 458  ☑ 458 
Q 301-85  T 216  ☑ 216 
Q 490-356 T 134  ☑ 134 
Q 68+939  T 1007 ☑ 1007
Q 521+25  T 546  ☑ 546 
Q 79+390  T 469  ☑ 469 

--------------------------------------------------
Iteration 71
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 101us/step - loss: 3.3709e-04 - acc: 1.0000 - val_loss: 0.0585 - val_acc: 0.9843
Q 948-423 T 525  ☑ 525 
Q 898-94  T 804  ☑ 804 
Q 64+83   T 147  ☑ 147 
Q 554-95  T 459  ☑ 459 
Q 939+193 T 1132 ☑ 1132
Q 556-361 T 195  ☑ 195 
Q 64+559  T 623  ☑ 623 
Q 185-77  T 108  ☑ 108 
Q 352-5   T 347  ☑ 347 
Q 675+570 T 1245 ☑ 1245

--------------------------------------------------
Iteration 72
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/3600

36000/36000 [==============================] - 4s 103us/step - loss: 0.0433 - acc: 0.9865 - val_loss: 0.0846 - val_acc: 0.9759
Q 668+143 T 811  ☑ 811 
Q 744+12  T 756  ☑ 756 
Q 993+629 T 1622 ☑ 1622
Q 148-2   T 146  ☑ 146 
Q 34+774  T 808  ☑ 808 
Q 643-76  T 567  ☑ 567 
Q 105-68  T 37   ☑ 37  
Q 216+14  T 230  ☑ 230 
Q 741-3   T 738  ☑ 738 
Q 81+514  T 595  ☑ 595 

--------------------------------------------------
Iteration 85
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 102us/step - loss: 0.0042 - acc: 0.9990 - val_loss: 0.0708 - val_acc: 0.9802
Q 152-50  T 102  ☑ 102 
Q 436-81  T 355  ☑ 355 
Q 78+62   T 140  ☑ 140 
Q 510-206 T 304  ☒ 303 
Q 669-662 T 7    ☒ 6   
Q 491+51  T 542  ☑ 542 
Q 86+503  T 589  ☒ 599 
Q 773+40  T 813  ☑ 813 
Q 634+37  T 671  ☑ 671 
Q 626+9   T 635  ☑ 635 

--------------------------------------------------
Iteration 86
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [=====

36000/36000 [==============================] - 4s 108us/step - loss: 3.7811e-04 - acc: 1.0000 - val_loss: 0.0580 - val_acc: 0.9841
Q 979-5   T 974  ☑ 974 
Q 555-155 T 400  ☑ 400 
Q 439-134 T 305  ☑ 305 
Q 261-93  T 168  ☑ 168 
Q 901-231 T 670  ☑ 670 
Q 255+343 T 598  ☒ 698 
Q 81+281  T 362  ☑ 362 
Q 212+545 T 757  ☑ 757 
Q 723+3   T 726  ☑ 726 
Q 239+51  T 290  ☑ 290 

--------------------------------------------------
Iteration 99
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 4s 107us/step - loss: 3.6328e-04 - acc: 1.0000 - val_loss: 0.0585 - val_acc: 0.9840
Q 860-457 T 403  ☑ 403 
Q 547+351 T 898  ☑ 898 
Q 945-284 T 661  ☑ 661 
Q 329-85  T 244  ☑ 244 
Q 369+503 T 872  ☑ 872 
Q 985-768 T 217  ☑ 217 
Q 872-219 T 653  ☑ 653 
Q 918-25  T 893  ☑ 893 
Q 79+390  T 469  ☑ 469 
Q 599+791 T 1390 ☒ 1380


In [34]:
model.save_weights('layer_1.h5')
# model.load_weights('my_model_weights.h5')

### Testing

In [35]:
evaluated_loss, evaludated_accuracy = model.evaluate(x=test_x, y=test_y)

120000/120000 [==============================] - 41s 339us/step


In [36]:
print("testing loss:{}, testing accuracy:{}".format(evaluated_loss, evaludated_accuracy))

testing loss:0.05609151270463287, testing accuracy:0.9845333333333334
